In [6]:
# Restart Kernel after install so modules can be imported
%cd /project_ghent/GANs_Pose_Estimation_on_Art_Collection
%sudo bash "commands/Style Transfer/install_UGATIT.sh"
%pip install -r "requirements.txt"
%cd src

/project_ghent/GANs_Pose_Estimation_on_Art_Collection
Update pip
Looking in indexes: https://download.pytorch.org/whl/cu118
Already up to date.
running develop
/opt/conda/lib/python3.11/site-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/opt/conda/lib/python3.11/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid 

In [1]:
## Change this value when you get a CUDA out of memory error
%env PYTORCH_CUDA_ALLOC_CONF max_split_size_mb:512

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512


In [2]:
import sys
import os
sys.path.append(f"{os.getcwd()}/src")
print(os.getcwd())
from style_transfer.networks import UGATIT, UGATITConfig

def train():
    config = UGATITConfig.create("style_transfer/config/ugatit.yaml")
    # Dataset
    config.defrost()
    config.dataset = "BaroqueStyleTraining"
    config.data_dir = "../../Datasets"
    config.result_dir = "../../Results"
    # Training
    config.iteration = 1000000
    config.batch_size = 1
    config.print_freq = 1000
    config.save_freq = 1000
    config.freeze()
    network = UGATIT(config)
    network.train()
    print("Finished!")

d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\src


KeyboardInterrupt: 

In [ ]:
import cv2
from PIL import Image
import torch
from torchvision.transforms import transforms

def transform(path, direction="AtoB"):
    config = UGATITConfig.create("src/style_transfer/config/ugatit.yaml")
    network = UGATIT(config)
    network.loadModel("style_transfer/model")
    with open(path, 'rb') as file:
        image = Image.open(file)
        image.convert("RGB")
        
    image = transforms.ToTensor()(image)
    image = transforms.Resize((256, 256))(image)
    image = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = torch.stack((image,))
    image = image.to("cuda")

    if direction == "AtoB":        
        image = network.transformFromPhotographicToArtistic(image)
    elif direction == "BtoA":
        image = network.transformFromArtisticToPhotographic(image)
    
    image = image[0] * 0.5 + 0.5
    image = image.detach().cpu().numpy()
    image = image.transpose(1,2,0)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(config.result_dir, "test_ugatit.png"), image * 255.0)

In [ ]:
train()
# transform("../../Datasets/human-art/images/2D_virtual_human/oil_painting/000000000000.jpg", "BtoA")